# Exercise
Calculate the total number of bookings for each facility.

In [27]:
import psycopg2
import pandas as pd

# The connection details (user, password, host) below are placeholders.
# Your coach will provide you with the actual connection details for your exercises.

conn_params = {
    'host': 'etl-relational-database.cfmnnswnfhpn.eu-west-2.rds.amazonaws.com',
    'dbname': 'postgres',
    'user': 'readonly_user',
    'password': 'you-can-only-read'
}

In [28]:
def fetch_data(query):
    with psycopg2.connect(**conn_params) as conn:
        with conn.cursor() as cur:
            cur.execute(query)
            data = cur.fetchall()
            colnames = [desc[0] for desc in cur.description]
    return data, colnames

In [29]:
# Extract bookings data
query = "SELECT b.facid, f.name, COUNT(b.facid) as total_bookings FROM bookings b JOIN facilities f ON b.facid = f.facid GROUP BY b.facid, f.name ORDER BY b.facid;"
bookings_data, colnames = fetch_data(query)
bookings_df = pd.DataFrame(bookings_data, columns=colnames)
bookings_df

,facid,name,total_bookings
0,0,Tennis Court 1,408
1,1,Tennis Court 2,389
2,2,Badminton Court,383
3,3,Table Tennis,403
4,4,Massage Room 1,629
5,5,Massage Room 2,111
6,6,Squash Court,440
7,7,Snooker Table,444
8,8,Pool Table,837


In [30]:
def execute_query_postgresql(conn_string, query):
    with psycopg2.connect(conn_string) as conn:
        with conn.cursor() as cur:
            cur.execute(query)
            conn.commit()

# Define the connection string for your analytical database
etl_bites_conn_string = "dbname='etl_bites' user='jackdench' host='localhost' port='5432'"

# SQL query to create a new table for storing total booking duration per facility
create_total_bookings_table = '''
CREATE TABLE total_bookings (
    FacilityID INTEGER NOT NULL,
    FacilityName TEXT,
    NumberOfBookings INTEGER
);
'''

# Execute the query to create the table
execute_query_postgresql(etl_bites_conn_string, create_total_bookings_table)

In [31]:
def insert_data(conn_string, table_name, data, columns):
    with psycopg2.connect(conn_string) as conn:
        with conn.cursor() as cur:
            for row in data.itertuples(index=False):
                insert_query = f"INSERT INTO {table_name} ({', '.join(columns)}) VALUES ({', '.join(['%s'] * len(columns))});"
                cur.execute(insert_query, row)
            conn.commit()

# Insert the transformed data into the analytical database
insert_data(etl_bites_conn_string, 'total_bookings', bookings_df, ['FacilityID', 'FacilityName', 'NumberOfBookings'])